In [ ]:
import sys

!"{sys.executable}" -m pip install --upgrade pip

In [ ]:
#Load all libraries

# Azure SDK for storage and identity
!"{sys.executable}" -m pip install azure-storage-blob azure-identity
# Data manipulation
!"{sys.executable}" -m pip install pandas numpy sklearn
# Visualization
!"{sys.executable}" -m pip install matplotlib
# Tooling to perform ARIMA forecasts
!"{sys.executable}" -m pip install pmdarima
# Tooling for PySpark
!"{sys.executable}" -m pip install pyspark
# Needed to run the notebook
!"{sys.executable}" -m pip install jupyter notebook
# KQL Magic
!"{sys.executable}" -m pip install Kqlmagic --no-cache-dir  --upgrade


In [ ]:
from azure.identity import DefaultAzureCredential

#credential = DefaultAzureCredential(True)

In [ ]:
#from azure.storage.blob import BlockBlobService
from datetime import datetime, timedelta
from azure.storage.blob import BlobServiceClient, generate_account_sas, generate_container_sas, ResourceTypes, AccountSasPermissions, ContainerSasPermissions

# Name of the Azure Storage account
azure_storage_account = '#STORAGE_ACCOUNT_NAME#'
# Name of the Container holding the data
azure_storage_container = 'loganalytics'
# Base path for the blobs within the container
azure_storage_path = ''

storage_account_url = "https://{}.blob.core.windows.net".format(azure_storage_account)

permission = ContainerSasPermissions(read=True, write=True, delete=True, 
                                 list=True,delete_previous_version=True, tag=True)

sas_token = generate_container_sas(
    account_name="#STORAGE_ACCOUNT_NAME#",
    account_key="#STORAGE_ACCOUNT_KEY#",
    container_name=azure_storage_container,
    #resource_types=ResourceTypes(container=True),
    permission=permission,
    expiry=datetime.utcnow() + timedelta(hours=1)
)

storage_client = BlobServiceClient(account_url="https://#STORAGE_ACCOUNT_NAME#.blob.core.windows.net", credential=sas_token)

#block_blob_service = BlockBlobService(account_name='cjgstats', account_key='5/c6LtMdJWfaj9RS8kFGGe179Ug3Bhe3PxngyS4crD7ctrs03M28bvQi3h7CNnAgh8qINcdczXzYMoiAE7mtkA==') 



In [ ]:
import json
import pandas as pd
from datetime import datetime, timedelta

def extract_requests_from_container(client, blob_path, container_name, start_time=None, end_time=None):

    container = client.get_container_client(container_name)

    blob_list = container.list_blobs(blob_path)

    combined_results = []

    for blob in blob_list:

        body = container.download_blob(blob.name).readall().decode('utf8')

        for request_string in body.split('\n'):
            try:
                request = json.loads(request_string)
                combined_results.append(request)
            except Exception as e:
                print(e)
                continue
    return combined_results

data = extract_requests_from_container(storage_client, azure_storage_path, azure_storage_container, 
datetime.utcnow() - timedelta(hours=3), datetime.utcnow())

df = pd.DataFrame(data)

print(df)